In [3]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing

preproc = SkinLesionPreprocessing()
examples = SegExamples()

In [41]:
def gabor_calculate(im):
    df = pd.DataFrame()
    img2 = im.reshape(-1) # add the pixel values of the org image as feature.
    df['org_img'] = img2

    ksize = 3 # Keep it small not to miss small features
    sigma = 1 # Keep it small not to miss small features
    lamda = 1*np.pi/4
    gamma = 1 # if it is 1 meaning that it is looking for circular features, 0 otherwise.
    phi = 0
    num = 1
    for d in range(1,4,1):
        gabor_label = "Gabor"+str(num)
        theta = 1 * np.pi/d
        kernel = cv2.getGaborKernel((ksize,ksize), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F)
        fimg = cv2.filter2D(im, cv2.CV_8UC3, kernel)
        filtered_img = fimg.reshape(-1) # 1d. add filtered image as feature
        df[gabor_label] = filtered_img
        num += 1
    return df

In [42]:
def get_gabor():
    labels = []
    features = pd.DataFrame()
    for i in tqdm(range(examples.seg_examples_df.shape[0])):
        if examples[i]['problem'] != 'binary':
            continue
        labels.append(examples[i]['label'])
        image = examples[i]['img']
        image = preproc.preprocess(image) # the img uploaded as numpy array.
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # by converting to gray we dont lose information in gabor filters
        dataframe = gabor_calculate(gray)
        ## Take mean, varience, skenwness, kurtosis and minimum for each filter
        df1=dataframe.mean().to_frame().T
        df2=dataframe.var().to_frame().T
        df3=dataframe.skew().to_frame().T
        df4=dataframe.kurtosis(axis=0).to_frame().T
        df5=dataframe.min().to_frame().T

        frames = [df1, df2, df3, df4, df5]
        result = pd.concat(frames, axis=1, join='inner')
        features = pd.concat([features, result])
    return features

In [43]:
get_gabor()

100%|██████████| 68/68 [02:48<00:00,  2.48s/it]


,org_img,Gabor1,Gabor2,Gabor3,org_img,Gabor1,Gabor2,Gabor3,org_img,Gabor1,Gabor2,Gabor3,org_img,Gabor1,Gabor2,Gabor3,org_img,Gabor1,Gabor2,Gabor3
0,134.314310,229.084444,229.086946,54.777594,1537.089684,4125.636717,4125.125494,259.954987,-2.195676,-2.384671,-2.384544,-2.164329,3.387853,4.009561,4.008645,3.343884,9,15,18,0
0,193.310793,254.536637,254.534917,78.838417,375.592504,10.877709,10.840185,68.326036,-1.603265,-8.971034,-8.916911,-1.577447,2.166691,92.883545,91.641167,2.333433,103,190,191,23
0,143.568892,246.585993,246.585750,58.551508,312.415628,665.150810,665.162637,54.738595,-2.738267,-3.608492,-3.608612,-2.555577,7.886986,12.666157,12.667527,7.249848,46,86,86,14
0,109.975288,200.219579,200.223545,44.851025,501.281377,1637.320379,1635.899263,89.636193,-0.443727,-0.471928,-0.471936,-0.433942,-1.153317,-1.178725,-1.179926,-0.953941,54,97,101,0
0,135.561782,234.944149,234.942180,55.286222,486.745605,553.535439,553.581680,84.862083,0.582069,-1.182888,-1.181661,0.510292,0.612725,0.591476,0.585628,0.585316,73,136,136,15
0,154.146246,237.938842,237.937170,62.866251,1322.584953,1075.632035,1075.812282,224.097868,-0.333659,-1.797734,-1.797691,-0.350004,-0.460314,1.848268,1.848412,-0.425981,55,101,101,3
0,151.326294,238.690900,238.694826,61.715922,1069.053225,899.332644,899.200194,183.347179,-0.302181,-1.682206,-1.682684,-0.299942,-0.495492,1.427917,1.429027,-0.415784,69,127,126,0
0,159.278072,253.185167,253.185351,64.958410,126.466298,106.663104,106.666495,23.138656,-3.171551,-6.312571,-6.313044,-2.771716,13.246593,40.370674,40.375465,11.071691,85,157,156,32
0,171.696728,244.417043,244.417098,70.023408,1380.774150,870.362946,870.311317,233.428752,-1.012982,-2.971476,-2.971395,-1.008274,0.558069,7.916063,7.915505,0.558098,40,79,75,0
0,170.665770,250.995908,250.998451,69.603560,771.310899,223.609677,223.789214,133.552379,-0.474697,-5.044063,-5.045632,-0.498898,0.184928,28.659977,28.668975,0.243373,51,96,94,12
